%%

In [1]:
# TODO: add comments. clean up code. put the py into ipynb and see if the figure will build
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import g_key

In [2]:
# natively display more pd info in pycharm
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# endregion

In [5]:
df = pd.read_csv(r'../WeatherPy/cities_2.csv')

In [6]:
df.rename(columns={'Unnamed: 0': 'city'}, inplace=True)
df.head()

,city,lat,long,max_temp,humidity,wind_speed,clouds,country,dt,is_south
0,gushikawa,26.3544,127.8686,70.83,72,4.629214,34,Japan,1635184942,0
1,punta arenas,-53.1500,-70.9167,42.82,70,7.350819,40,Chile,1635184753,1
2,west wendover,40.7391,-114.0733,60.94,50,2.970153,100,United States of America,1635184887,0
3,cape town,-33.9258,18.4232,68.34,63,0.832637,13,South Africa,1635184933,1
4,busselton,-33.6500,115.3333,41.04,71,1.217887,73,Australia,1635184942,1


In [7]:
gmaps.configure(g_key)

In [8]:
locs = df[['lat', 'long']]
humidity = df.humidity

In [9]:
fig_layout = {
        'width':'1200px',
        'height':'600px',
        'border':'2px solid black',
        'padding':'2px',
        'margin':'0 auto 0 auto'
}

In [10]:
fig = gmaps.figure(layout=fig_layout)

In [11]:
heatmap = gmaps.heatmap_layer(
        locs,
        weights=humidity,
        max_intensity=df.humidity.max(),
        dissipating=False,
        point_radius=5
)

In [13]:
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='2px solid black', height='600px', margin='0 auto 0 auto', padding='2px', wi…

select decent weather

In [ ]:
temp_mask = df['max_temp'].between(70, 90)
humid_mask = df['humidity'].between(50, 90)
cloud_mask = df['clouds'] < 8

mask based on conditions

In [ ]:
masked_df = df[temp_mask & humid_mask & cloud_mask]

# check for nulls
print(len(masked_df))

In [ ]:
masked_df.isna().sum()
len(masked_df)

find hotels within 5e3 m of my locations

In [ ]:
hotels = []

In [ ]:
for i in masked_df['city']:
    lat = masked_df['lat']
    long = masked_df['long']
    coords = list(zip(lat.astype(str), long.astype(str)))
    coords = [', '.join(coord) for coord in coords]
    param_dict = {
            'location': coords,
            'radius': 5000,
            'type': 'lodging',
            'key': g_key
    }
    url_base = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    try:
        resp = requests.get(url_base, param_dict).json()
        outputs = resp['results'][0]['name']
        hotels.append(outputs)
    except:
        hotels.append(np.nan)
        print(f'Unable to find {i}.')

In [ ]:
hotel_names = [outputs[i]['name'] for i in range(len(masked_df.city))]
print(len(hotel_names) == len(masked_df.city))
masked_df['hotel'] = hotel_names

In [ ]:
masked_df.reset_index(drop=True)

Using the template add the hotel marks to the heatmap

info_box_template = 
<br>
<dl><br>
<dt>Name</dt><dd>{hotel}</dd><br>
<dt>City</dt><dd>{city}</dd><br>
<dt>Country</dt><dd>{country}</dd><br>
</dl><br>
<br>
Store the DataFrame Row<br>
NOTE: be sure to update with your DataFrame name

In [ ]:
hotel_info = [info_box_template.format(**row) for index, row in masked_df.iterrows()]
locations = masked_df[["lat", "long"]]

In [ ]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)